<img src="../images/storm-animation.gif" width=600 alt="TRACER Animation"></img>
# Plotting Data from a Field Campaign (TRACER)

## Overview

Within this notebook, we will cover:

1. How to access data from the Atmospheric Radiation Measurment (ARM) user facility
1. How to setup a workflow to plot both cross sections (RHIs) and horizontal scans (PPIs)

## Prerequisites
| Concepts | Importance | Notes |
| --- | --- | --- |
| [Matplotlib Basics](https://foundations.projectpythia.org/core/matplotlib/matplotlib-basics.html) | Required | Basic plotting |
| [Py-ART Basics](../foundations/pyart-basics) | Required | IO/Visualization |

---
## Imports

In [ ]:
import glob
import os
from pathlib import Path

import act
import imageio.v2 as imageio
import matplotlib
import matplotlib.pyplot as plt
import pyart

## Grab Data from ARM

One of the better cases was from June 2, 2022, where several cold pools and single-cell storms traversed through the domain.

### The Tracking Aerosol Convection Interactions ExpeRiment (TRACER) Field Campaign

Data is available from the Atmospheric Radiation Measurment user facility, which helped to lead the TRACER field campaign in Houston, Texas.

The data are available from the ARM data portal (https://adc.arm.gov/).

We are interested in the C-band radar, which is utilizing a cell-tracking algorithm, with the datastream
* **`houcsapr2cfrS2.a1`**

### Use the ARM Live API to Download the Data, using ACT

The Atmospheric Data Community Toolkit (ACT) has a helpful module to interface with the data server:
* [Download Data API](hhttps://arm-doe.github.io/ACT/API/generated/act.discovery.download_arm_data.html)

In [ ]:
act.discovery.download_arm_data?

### Setup our Download Query
Before downloading our data, we need to make sure we have an ARM Data Account, and ARM Live token. Both of these can be found using this link:
- [ARM Live Signup](https://adc.arm.gov/armlive/livedata/home)

Once you sign up, you will see your token. Copy and replace that where we have `arm_username` and `arm_password` below.

In [ ]:
arm_username = os.getenv("ARM_USERNAME")
arm_password = os.getenv("ARM_PASSWORD")

datastream = "houcsapr2cfrS2.a1"

start_date = "2022-06-02T11:30:00"
end_date = "2022-06-02T11:40:00"

print(len(arm_username), len(arm_password))

In [ ]:
june2_csapr_files = act.discovery.download_arm_data(arm_username,
                                                arm_password,
                                                datastream,
                                                start_date,
                                                end_date,
                                               )

## Read in and Plot the Data
Before following running the next cells, make sure you have created the following directories:
- `quicklooks/ppi`
- `quicklooks/rhi`
- `quicklooks/vpt`

In [ ]:
!mkdir quicklooks
!mkdir quicklooks/rhi
!mkdir quicklooks/ppi
!mkdir quicklooks/vpt

### Loop through and plot
We read in the data, check the scan type, and plot a basic `RadarDisplay` which will automatically detect whether the plot is a vertical cross section (RHI or VPT), or a horizontal scan (PPI).

This offers a solid "quick look", or initial visualization of the data.

In [ ]:
for file in june2_csapr_files:
    radar = pyart.io.read(file)
    print(radar.scan_type)
    display = pyart.graph.RadarDisplay(radar)
    display.plot("reflectivity", 0)
    plt.savefig(f"{radar.scan_type}_{Path(file).stem}.png", dpi=200)
    plt.show()
    plt.close() 

### Refine our Plot, Plot Velocity
Let's focus on the vertical scans of the data, or the RHIs.

You'll notice that we had some cells around 60 km from the radar, with the vertical axis less than 6 km.

Let's reflect that in the plots!

#### Customize our plot look
Before we plot, we can change the size of our font, and style using the following parameters:

In [ ]:
font = {'family' : 'serif',
        'weight' : 'bold',
        'size'   : 16}

matplotlib.rc('font', **font)

#### Apply our Plotting Loop
We:
- Check to see if the scan is an RHI
- Plot the reflectivity on one subplot
- Plot velocity on the other
- Save our plots

In [ ]:
for file in june2_csapr_files:
    radar = pyart.io.read(file)
    if radar.scan_type == 'rhi':
        fig = plt.figure(figsize=(20,8))
        display = pyart.graph.RadarDisplay(radar)
        ax = plt.subplot(121)
        display.plot("reflectivity",
                     0,
                     ax=ax,
                     vmin=-20,
                     vmax=70)
        plt.xlim(62,68)
        plt.ylim(0, 6)
        
        ax2 = plt.subplot(122)
        display.plot("mean_doppler_velocity",
                     0,
                     ax=ax2, 
                     cmap='balance',
                     vmin=-15,
                     vmax=15)
        plt.xlim(62,68)
        plt.ylim(0, 6)
        plt.savefig(f"{radar.scan_type}_{Path(file).stem}.png", dpi=200)
        plt.show()
        plt.close() 

### Create a GIF of the RHI images

In [ ]:
rhi_images = sorted(glob.glob(f"{radar.scan_type}*"))

In [ ]:
with imageio.get_writer('storm-animation.gif', mode='I') as writer:
    for filename in rhi_images:
        image = imageio.imread(filename)
        writer.append_data(image)

<img src="storm-animation.gif" width="750" align="center">

---
## Summary
Within this example, we walked through how to access ARM data from a field campaign in Texas, plot a quick look of the data, and refine our plots to investigate a storm!

### What's Next?
We will showcase other data workflow examples, including field campaigns in other regions and data access methods from other data centers.

## Resources and References
* [ARM Data Discovery](https://adc.arm.gov/discovery)
* [TRACER Field Campaign](https://www.arm.gov/research/campaigns/amf2021tracer)
* CSAPR Radar Data:
    - Bharadwaj, N., Collis, S., Hardin, J., Isom, B., Lindenmaier, I., Matthews, A., & Nelson, D. C-Band Scanning ARM Precipitation Radar (CSAPR2CFR). Atmospheric Radiation Measurement (ARM) User Facility. https://doi.org/10.5439/1467901
* Py-ART:
    - Helmus, J.J. & Collis, S.M., (2016). The Python ARM Radar Toolkit (Py-ART), a Library for Working with Weather Radar Data in the Python Programming Language. Journal of Open Research Software. 4(1), p.e25. DOI: http://doi.org/10.5334/jors.119
* ACT:
    - Adam Theisen, Ken Kehoe, Zach Sherman, Bobby Jackson, Alyssa Sockol, Corey Godine, Max Grover, Jason Hemedinger, Jenni Kyrouac, Maxwell Levin, Michael Giansiracusa (2022). The Atmospheric Data Community Toolkit (ACT). Zenodo. DOI: https://doi.org/10.5281/zenodo.6712343